In [1]:
import understand
import sys, traceback


def open_understand_db(path):
    try:
        db = understand.open(path)
    except understand.UnderstandError:
        traceback.print_exc(file=sys.stdout)
    return db

In [41]:
def lookup_imports(file_ents, import_prefix):
    imports = [(None, [])]
    for f in file_ents:
        import_refs = f.refs("import")
        for i in import_refs:
            imported_ent = i.ent()
            if imported_ent.longname().startswith(import_prefix):
                if f is not imports[-1][0]:
                    imports.append((f, []))
                imports[-1][1].append(imported_ent)
    del imports[0]
    return imports

In [44]:
def lookup_mentions(ents, subject):
    mentions = [(None, [])]
    for e in ents:
        try:
            lexer = e.lexer()
            for lexeme in lexer:
                if subject in lexeme.text().casefold():
                    if e is not mentions[-1][0]:
                        mentions.append((e, []))
                    mentions[-1][1].append(lexeme)
        except understand.UnderstandError:
            traceback.print_exc(file=sys.stdout)
    del mentions[0]
    return mentions

In [59]:
def analyze_jpos():
    db = open_understand_db("../data/jpos/jpos.udb")
    jpos_impl = db.lookup_arch("Directory Structure/main/java/org/jpos")
    jpos_files = jpos_impl.ents(True)
    bc_imports = lookup_imports(jpos_files, "org.bouncycastle")
    bc_metrics = []
    for bci in bc_imports:
        bc_metrics.append((bci[0], bci[0].metric(["CountLine", "SumCyclomatic"])))
    #bc_mentions = lookup_mentions(jpos_files, "bouncycastle")
    #bc_mentions1 = [(m[0].longname(), 
    #                 [(l.token(), l.ent().kind() if l.ent() else None, 
    #                   l.ent().longname() if l.ent() else None, l.text()) 
    #                  for l in m[1]]) for m in bc_mentions]
    
    from IPython.display import display
    
    display(bc_imports)
    display(bc_metrics)
    #display(bc_mentions)
    #display(bc_mentions1)

In [60]:
analyze_jpos()

[(jJPOS:/jpos/src/main/java/org/jpos/security/SystemSeed.java,
  [jorg.bouncycastle.util.encoders.Base64@]),
 (jJPOS:/jpos/src/main/java/org/jpos/util/PGPHelper.java,
  [jorg.bouncycastle.openpgp@,
   jorg.bouncycastle.bcpg.ArmoredInputStream@,
   jorg.bouncycastle.jce.provider.BouncyCastleProvider@,
   jorg.bouncycastle.openpgp.operator.KeyFingerPrintCalculator@,
   jorg.bouncycastle.openpgp.operator.bc.BcKeyFingerprintCalculator@,
   jorg.bouncycastle.openpgp.operator.jcajce.JcaPGPContentVerifierBuilderProvider@])]

[(jJPOS:/jpos/src/main/java/org/jpos/security/SystemSeed.java,
  {'CountLine': 66, 'SumCyclomatic': 5}),
 (jJPOS:/jpos/src/main/java/org/jpos/util/PGPHelper.java,
  {'CountLine': 213, 'SumCyclomatic': 34})]